In [ ]:
#
# Representation using splines for edges
#
import polars   as pl
import networkx as nx
import rtsvg
rt = rtsvg.RACETrack()

edges_filename  = '../../data/stanford/facebook/348.edges'
layout_filename = '../../data/stanford/facebook/348.rt_layout.parquet'

_lu_ = {'fm':[], 'to':[]}
for _edge_ in open(edges_filename, 'rt').read().split('\n'):
    if _edge_ == '': continue
    _split_     = _edge_.split()
    _fm_, _to_  = int(_split_[0]), int(_split_[1])
    _lu_['fm'].append(_fm_), _lu_['to'].append(_to_)
df  = pl.DataFrame(_lu_)
g   = rt.createNetworkXGraph(df, [('fm','to')])
pos = nx.spring_layout(g)
#rt.link(df, [('fm','to')], pos, w=800, h=600, link_shape='curve', link_size=1)

In [ ]:
df = df.with_columns(pl.col('fm').map_elements(lambda x: pos[x][0], return_dtype=pl.Float32).alias('x0'),
                     pl.col('fm').map_elements(lambda y: pos[y][1], return_dtype=pl.Float32).alias('y0'),
                     pl.col('to').map_elements(lambda x: pos[x][0], return_dtype=pl.Float32).alias('x1'),
                     pl.col('to').map_elements(lambda y: pos[y][1], return_dtype=pl.Float32).alias('y1'))
df = df.with_columns((pl.col('x1') - pl.col('x0')).alias('dx'),
                     (pl.col('y1') - pl.col('y0')).alias('dy'))
df = df.with_columns((pl.col('dx')**2 + pl.col('dy')**2).sqrt().alias('l'))
df = df.with_columns((pl.col('dx')/pl.col('l')).alias('u'),
                     (pl.col('dy')/pl.col('l')).alias('v'))
df = df.with_columns((-1.0 * pl.col('v')).alias('pu'),
                             pl.col('u') .alias('pv'))
df = df.with_columns(((pl.col('x1') - pl.col('x0')) * (1.0/3.0) + pl.col('x0') + pl.col('pu')*0.01).alias('xa'),
                     ((pl.col('y1') - pl.col('y0')) * (1.0/3.0) + pl.col('y0') + pl.col('pv')*0.01).alias('ya'),
                     ((pl.col('x1') - pl.col('x0')) * (2.0/3.0) + pl.col('x0') + pl.col('pu')*0.01).alias('xb'),
                     ((pl.col('y1') - pl.col('y0')) * (2.0/3.0) + pl.col('y0') + pl.col('pv')*0.01).alias('yb'))
df